In [8]:
%matplotlib inline

## Solar Angle Calculation

To calculate solar time, first thing to do is to calculate approximate solar time based on local time, time zone and daylight saving. Since the local time on your watch is not actual local solar time, but the solar time of the local time zone meridian.
$$ AST = LST + ET + 4 ( LSM - LON ) $$
where $AST$ is the approximate solar time in minutes, $LST$ is the local standard time in minutes, $LSM$ is the meridian of local standard time in degrees and $LON$ is the local longitude. 4 is the minutes of time required for 1 degree of earth rotation. ET is the equation of time in minutes, obtained by:
$$ ET(n) = 9.87sin(4\pi\frac{n-81}{364})-7.3cos(2\pi\frac{n-81}{364})-1.5sin(2\pi\frac{n-81}{364}) $$
n is the day of year from 1 to 365.
Now to create those to functions in python:

In [46]:
from math import pi, sin, cos, asin, acos
def EquationTime(n):
    if n <1.0 or n > 365.0:
        raise ValueError("n is the day number between 1 and 365")
    else:
        b = 2*math.pi*(n-81.0)/364.0
        return 9.87*sin(2*b)-7.3*cos(b)-1.5*sin(b)

    
def ActualSolarTime(LST, ET, LSM, LON):
    #check if LST is valid
    if LST<0.0 or LST>1440.0:
        raise ValueError("Local Standard Time needs to be in minutes and between 0 and 1440")
    #check if ET is valid
    if ET<-3.563463668700962:
        raise ValueError("Equation Time is invalid")
    #check if LSM is valid
    if LSM > 180 or LSM < -180:
        raise ValueError("Local Standard Time Meridian is within -180 and 180 degrees")
    if LON > 180 or LON < -180:
        raise ValueError("Local Longitude is within -180 and 180 degrees")
    return LST+ET+4*(LSM-LON)

In [47]:
EquationTime(1)

-3.563463668700962

In [48]:
from library.Location import Location
Ottawa = Location("Ottawa", -75.6919, 45.4214)
print(Ottawa.timezone)
print(Ottawa.lsm)

-5.0
-75.0


In [49]:
import numpy as np
day = 1
et = EquationTime(day)
lst = np.arange(24*60)
ast = []
for i in range(0, 24*60):
    ast.append(ActualSolarTime(lst[i],et,Ottawa.lsm,Ottawa.longitude))

print(ast[:10])

[-0.79586366870094638, 0.20413633129905362, 1.2041363312990536, 2.2041363312990536, 3.2041363312990536, 4.2041363312990541, 5.2041363312990541, 6.2041363312990541, 7.2041363312990541, 8.2041363312990541]


Next is to express the solar time in degrees from the sun's vertical position (noon) to adjust the difference due to time zone:
$$ h = (AST-12.00*60)*0.25 $$

declination angle, the sun's declination due to earth's rotation, is given by:

$$ \delta = sin^{-1}(sin(\frac{23.45}{180}\pi)sin(2\pi\frac{284+n}{365}))$$

altitude angle can be calculated by:
$$ \alpha = sin^{-1}(sin\delta sin\phi + cos\delta cos\phi cos(h))$$
$\phi$ is the latitude

zenith, $\theta$, is then
$$\theta = \frac{1}{2}\pi - \alpha$$

azimuth angle can be calculated by:
$$ azi = cos^{-1}(\frac{sin\alpha cos\phi - sin\delta}{cos\alpha cos\phi})$$

So to create this sun class in python:

In [51]:
class Sun:
    day = 0
    
    def __init__(self, lat, day, ast):
        #latitude of location in rediant
        self.latitude = lat/180.0*pi
        self.changeDay(day)
        self.updateAngles(ast)
    
    def changeDay(self,day):
        day = day
        self.declination = asin(sin(23.45/180*pi)*sin(2*pi*(284+day)/365))
        
    def updateAngles(self,ast):
        self.h = (ast-12.00*60)*0.25
        self.altitude = asin(sin(self.declination)*sin(self.latitude) + 
                                  cos(self.declination)*cos(self.latitude)*self.h)
        self.zenith = pi/2 - self.altiude
        self.azimuth = acos((sin(self.altitude)*cos(self.latitude)-sin(self.declination))/(cos(self.altitude)*cos(self.latitude)))